In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import friedmanchisquare
from scikit_posthocs import posthoc_nemenyi_friedman as nemenyi

In [2]:
df = pd.read_csv("/home/dobraczka/Downloads/git/er-embedding-benchmark/data/all_results.csv",header=0,sep=",")
df["dataset"] = df["dataset"].str.replace(r"-721_5fold-\d","")
rich_graph_datasets = {"D_W_15K_V1", "D_W_15K_V2", "D_Y_15K_V1", "D_Y_15K_V2", "EN_DE_15K_V1", "EN_DE_15K_V2", "EN_FR_15K_V1", "EN_FR_15K_V2"}
rich_df = df[df["dataset"].isin(rich_graph_datasets)]
shallow_df = df[~df["dataset"].isin(rich_graph_datasets)]

In [3]:
# creates a df with mean value of folds and all values per feature variant with all classifiers and datasets as rows
def get_expanded_df(df, removed_dim_name, fixed_value, levels):
    aggregated = df[df[removed_dim_name]==fixed_value].groupby(levels).mean()
    #level_zero = df["model_name"].unique()
    #level_one = df["vector_name"].unique()
    level_zero = df[levels[0]].unique()
    level_one = df[levels[1]].unique()
    variant_cols = []
    row_names = None
    for f in level_one:
        tmp_names = None
        tmp_col = None
        for m in level_zero:
            tmp = aggregated.loc[(m,f)]["test_f1"]
            tmp = tmp.add_suffix("_" + m)
            tmp.name = f
            if tmp_names is None:
                tmp_names = set(tmp.index.to_list())
            else:
                tmp_names = tmp_names | set(tmp.index.to_list())
            if tmp_col is None:
                tmp_col = tmp
            else:
                tmp_col = tmp_col.append(tmp)
        if row_names is None:
            row_names = tmp_names
        else:
            row_names = row_names & tmp_names
        variant_cols.append(tmp_col)
    for i in range(0, len(variant_cols)):
        variant_cols[i] = variant_cols[i].loc[row_names]
    expanded = pd.DataFrame(variant_cols).T
    expanded.sort_index(inplace=True)
    return expanded.round(2)

In [4]:
def highlight_significant(val):
    color = 'green' if val < 0.05 and val > 0.0 else 'black'
    return 'color: %s' % color

In [5]:
def stylize(df):
    cm = sns.cubehelix_palette(5, as_cmap=True)
    return df.style.background_gradient(cmap=cm, axis=1)

In [6]:
def statistical_test(df, features=True):
    #dont know how to better provide the arguments without it being interpreted as single array
    if features:
        statistic, pvalue = friedmanchisquare(df['OnlyEmb'], df['OnlySim'], df['OnlySimNormalized'], df['SimAndEmb'], df['SimAndEmbNormalized'])
    else:
        statistic, pvalue = friedmanchisquare(df.iloc[0],df.iloc[1],df.iloc[2],df.iloc[3],df.iloc[4],df.iloc[5],df.iloc[6],df.iloc[7])
    if pvalue < 0.05:
        print("There is a significant difference in the rank sums: ", pvalue)
        res = nemenyi(df)
        print("Significant pairwise differences (p < 0.05) are highlighted in green")
        return res.style.applymap(highlight_significant)
    else:
        print("No significant difference")
        return None

# Analysis of Rich Graph Datasets

Our hypothesis is that Knowledge Graph Embeddings (KGEs) perform well on datasets that with many edges between vertices. We also have the hypothesis that using both: KGEs and attribute similarities will have a beneficial effect on classifier performance. We gave several classifiers the following different features:

-__SimAndEmb__: Attribute similarities/distances and euclidean distance (of the two entities in embedding space)

-__OnlySim__: Only Attribute similarities/distances

-__SimAndEmbNormalized__: Attribute similarities and euclidean distance normalized between 0 and 1

-__OnlySimNormalized__: Attribute similarities normalized between 0 and 1

-__OnlyEmb__: Only euclidean distance
    
We do this for each different embedding method

## Analysis of Feature Input for RDGCN
To check if using embeddings in conjunction with similarities provides an improvement overall we sum the test_f1 value over all folds

In [7]:
rich_expanded = get_expanded_df(rich_df,"embed_model","RDGCN",["model_name","vector_name","dataset"])
stylized = stylize(rich_expanded)
stylized

,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
dataset,,,,,
D_W_15K_V1_MLP,0.82,0.79,0.79,0.75,0.73
D_W_15K_V1_decision tree,0.77,0.74,0.76,0.74,0.56
D_W_15K_V1_gaussian naive bayes,0.65,0.64,0.63,0.64,0.53
D_W_15K_V1_random forest 100,0.82,0.75,0.79,0.75,0.67
D_W_15K_V1_random forest 20,0.8,0.74,0.79,0.75,0.6
D_W_15K_V1_random forest 200,0.83,0.75,0.79,0.75,0.69
D_W_15K_V1_random forest 50,0.81,0.75,0.79,0.75,0.65
D_W_15K_V1_random forest 500,0.83,0.75,0.79,0.75,0.7
D_W_15K_V1_svc,0.86,0.8,0.68,0.64,0.79


Mean row wise rank (the lower the better):

In [8]:
rich_expanded.apply(lambda x: x.rank(ascending=False), axis=1).mean().sort_values()

SimAndEmb              1.458333
OnlySim                2.180556
SimAndEmbNormalized    3.069444
OnlySimNormalized      3.875000
OnlyEmb                4.416667
dtype: float64

Using embeddings AND similarity seems to improve the results.
Let's test if the difference is significant!

The Friedman test is designed to check if there is a significant difference in the rank sums.
If this is the case a nemenyi test is applied pairwise to check which differences are significant.

In [9]:
res = statistical_test(rich_expanded)
res

There is a significant difference in the rank sums:  1.3295139198792162e-37
Significant pairwise differences (p < 0.05) are highlighted in green


,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
SimAndEmb,-1,0.001,0.0482652,0.001,0.001
SimAndEmbNormalized,0.001,-1,0.00667029,0.0190035,0.001
OnlySim,0.0482652,0.00667029,-1,0.001,0.001
OnlySimNormalized,0.001,0.0190035,0.001,-1,0.239603
OnlyEmb,0.001,0.001,0.001,0.239603,-1


Values below 0.05 show a significant difference. So keeping the mean row wise ranks in mind:
SimAndEmb is significantly better than all other approaches.

# Analysis of Feature Input for MultiKE
To check if using embeddings in conjunction with similarities provides an improvement overall we sum the test_f1 value over all folds

In [10]:
rich_expanded = get_expanded_df(rich_df,"embed_model","MultiKE",["model_name","vector_name","dataset"])
stylized = stylize(rich_expanded)
stylized

,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
dataset,,,,,
D_W_15K_V1_MLP,0.88,0.9,0.83,0.75,0.84
D_W_15K_V1_decision tree,0.79,0.79,0.79,0.72,0.66
D_W_15K_V1_gaussian naive bayes,0.69,0.67,0.63,0.67,0.53
D_W_15K_V1_random forest 100,0.86,0.87,0.83,0.74,0.85
D_W_15K_V1_random forest 20,0.85,0.85,0.83,0.72,0.76
D_W_15K_V1_random forest 200,0.86,0.88,0.84,0.74,0.86
D_W_15K_V1_random forest 50,0.85,0.87,0.83,0.75,0.82
D_W_15K_V1_random forest 500,0.86,0.88,0.84,0.74,0.87
D_W_15K_V1_svc,0.87,0.86,0.68,0.7,0.88


Mean row wise rank (the lower the better):

In [11]:
rich_expanded.apply(lambda x: x.rank(ascending=False), axis=1).mean().sort_values()

SimAndEmb              1.791667
OnlySim                2.375000
SimAndEmbNormalized    2.611111
OnlyEmb                3.979167
OnlySimNormalized      4.243056
dtype: float64

Using embeddings AND similarities seems to improve the results.
Let's test if the difference is significant!

The Friedman test is designed to check if there is a significant difference in the rank sums.
If this is the case a nemenyi test is applied pairwise to check which differences are significant.

In [12]:
res = statistical_test(rich_expanded)
res

There is a significant difference in the rank sums:  9.188831028675198e-29
Significant pairwise differences (p < 0.05) are highlighted in green


,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
SimAndEmb,-1,0.0160839,0.174719,0.001,0.001
SimAndEmbNormalized,0.0160839,-1,0.896859,0.001,0.001
OnlySim,0.174719,0.896859,-1,0.001,0.001
OnlySimNormalized,0.001,0.001,0.001,-1,0.837036
OnlyEmb,0.001,0.001,0.001,0.837036,-1


Using similarities and embeddings is significantly better than all other approaches EXCEPT using only similarities, where the difference is not significant.

## Analysis of Feature Input for BootEA
To check if using embeddings in conjunction with similarities provides an improvement overall we sum the test_f1 value over all folds

In [13]:
shallow_expanded = get_expanded_df(shallow_df,"embed_model","BootEA",["model_name","vector_name","dataset"])
stylized = stylize(shallow_expanded)
stylized

,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
dataset,,,,,
abt-buy_MLP,0.92,0.78,0.41,0.83,0.3
abt-buy_decision tree,0.14,0.78,0.14,0.83,0.45
abt-buy_gaussian naive bayes,0.91,0.93,0.91,0.92,0.6
abt-buy_random forest 100,0.96,0.9,0.53,0.9,0.36
abt-buy_random forest 20,0.92,0.86,0.52,0.85,0.41
abt-buy_random forest 200,0.96,0.9,0.77,0.89,0.34
abt-buy_random forest 50,0.95,0.88,0.28,0.89,0.45
abt-buy_random forest 500,0.96,0.9,0.58,0.9,0.3
abt-buy_svc,0.53,0.76,0.58,0.78,0.4


Mean row wise rank (the lower the better):

In [14]:
shallow_expanded.apply(lambda x: x.rank(ascending=False), axis=1).mean().sort_values()

OnlySimNormalized      2.261905
SimAndEmb              2.285714
SimAndEmbNormalized    2.349206
OnlySim                3.388889
OnlyEmb                4.714286
dtype: float64

Interestingly normalization seems to be beneficial here

The Friedman test is designed to check if there is a significant difference in the rank sums.
If this is the case a nemenyi test is applied pairwise to check which differences are significant.

In [15]:
res = statistical_test(shallow_expanded)
res

There is a significant difference in the rank sums:  1.1015122578549892e-28
Significant pairwise differences (p < 0.05) are highlighted in green


,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
SimAndEmb,-1,0.9,0.001,0.9,0.001
SimAndEmbNormalized,0.9,-1,0.00208939,0.9,0.001
OnlySim,0.001,0.00208939,-1,0.001,0.001
OnlySimNormalized,0.9,0.9,0.001,-1,0.001
OnlyEmb,0.001,0.001,0.001,0.001,-1


Using normalized similarities and embeddings is significantly better than all other approaches except using not normalized similarities and embeddings

# Analysis of shallow Datasets

Our hypothesis here is that Knowledge Graph Embeddings (KGEs) don't significantly improve the results, because there is too little information in the embedding process

## Analysis of Feature Input for RDGCN
To check if using embeddings in conjunction with similarities provides an improvement overall we sum the test_f1 value over all folds

In [16]:
shallow_expanded = get_expanded_df(shallow_df,"embed_model","RDGCN",["model_name","vector_name","dataset"])
stylized = stylize(shallow_expanded)
stylized

,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
dataset,,,,,
abt-buy_MLP,0.84,0.88,0.64,0.79,0.26
abt-buy_decision tree,0.67,0.57,0.86,0.64,0.46
abt-buy_gaussian naive bayes,0.89,0.6,0.92,0.83,0.37
abt-buy_random forest 100,0.94,0.59,0.96,0.76,0.3
abt-buy_random forest 20,0.8,0.53,0.9,0.6,0.39
abt-buy_random forest 200,0.96,0.67,0.96,0.79,0.29
abt-buy_random forest 50,0.89,0.68,0.95,0.7,0.25
abt-buy_random forest 500,0.96,0.7,0.96,0.81,0.27
abt-buy_svc,0.95,0.86,0.93,0.79,0.16


Mean row wise rank (the lower the better):

In [17]:
rank_order = shallow_expanded.apply(lambda x: x.rank(ascending=False), axis=1).mean().sort_values()
# sort columns by rank to make nemenyi table easier to understand
shallow_expanded = shallow_expanded.reindex(rank_order.index, axis=1)
rank_order

OnlySim                1.658730
SimAndEmb              2.087302
OnlySimNormalized      2.777778
SimAndEmbNormalized    3.555556
OnlyEmb                4.920635
dtype: float64

Using only the similarities gives the best results.
Let's test if the difference is significant!

The Friedman test is designed to check if there is a significant difference in the rank sums.
If this is the case a nemenyi test is applied pairwise to check which differences are significant.

In [18]:
res = statistical_test(shallow_expanded)
res

There is a significant difference in the rank sums:  1.3705404565882028e-36
Significant pairwise differences (p < 0.05) are highlighted in green


,OnlySim,SimAndEmb,OnlySimNormalized,SimAndEmbNormalized,OnlyEmb
OnlySim,-1,0.541981,0.001,0.001,0.001
SimAndEmb,0.541981,-1,0.102075,0.001,0.001
OnlySimNormalized,0.001,0.102075,-1,0.0456482,0.001
SimAndEmbNormalized,0.001,0.001,0.0456482,-1,0.001
OnlyEmb,0.001,0.001,0.001,0.001,-1


Using only the similarities is significantly better than all other approaches EXCEPT using similarities and embeddings. Using similarities and embeddings is significantly better than using SimAndEmbNormalized and also than using just Embdeddings.

# Analysis of Feature Input for MultiKE
To check if using embeddings in conjunction with similarities provides an improvement overall we sum the test_f1 value over all folds

In [19]:
shallow_expanded = get_expanded_df(shallow_df,"embed_model","MultiKE",["model_name","vector_name","dataset"])
stylized = stylize(shallow_expanded)
stylized

,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
dataset,,,,,
dblp-scholar_MLP,0.99,0.99,0.99,0.99,0.52
dblp-scholar_decision tree,0.99,0.98,0.99,0.98,0.52
dblp-scholar_gaussian naive bayes,1,0.98,1,0.99,0.47
dblp-scholar_random forest 100,1,0.99,1,0.99,0.56
dblp-scholar_random forest 20,1,0.98,1,0.99,0.5
dblp-scholar_random forest 200,1,0.99,1,0.99,0.57
dblp-scholar_random forest 50,1,0.98,1,0.99,0.52
dblp-scholar_random forest 500,1,0.99,1,0.99,0.58
dblp-scholar_svc,1,0.99,1,0.99,0.5


Mean row wise rank (the lower the better):

In [20]:
rank_order = shallow_expanded.apply(lambda x: x.rank(ascending=False), axis=1).mean().sort_values()
# sort columns by rank to make nemenyi table easier to understand
shallow_expanded = shallow_expanded.reindex(rank_order.index, axis=1)
rank_order

SimAndEmb              2.111111
OnlySim                2.583333
OnlySimNormalized      2.611111
SimAndEmbNormalized    2.777778
OnlyEmb                4.916667
dtype: float64

Using embeddings AND similarities seems to improve the results.
Let's test if the difference is significant!

The Friedman test is designed to check if there is a significant difference in the rank sums.
If this is the case a nemenyi test is applied pairwise to check which differences are significant.

In [21]:
res = statistical_test(shallow_expanded)
res

There is a significant difference in the rank sums:  1.0103367980592226e-18
Significant pairwise differences (p < 0.05) are highlighted in green


,SimAndEmb,OnlySim,OnlySimNormalized,SimAndEmbNormalized,OnlyEmb
SimAndEmb,-1,0.686234,0.643932,0.381128,0.001
OnlySim,0.686234,-1,0.9,0.9,0.001
OnlySimNormalized,0.643932,0.9,-1,0.9,0.001
SimAndEmbNormalized,0.381128,0.9,0.9,-1,0.001
OnlyEmb,0.001,0.001,0.001,0.001,-1


Here using only embeddings is significantly outperformed by everything else. But there is no significant differnce between the other approaches.

## Analysis of Feature Input for BootEA
To check if using embeddings in conjunction with similarities provides an improvement overall we sum the test_f1 value over all folds

In [22]:
shallow_expanded = get_expanded_df(shallow_df,"embed_model","BootEA",["model_name","vector_name","dataset"])
stylized = stylize(shallow_expanded)
stylized

,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlySimNormalized,OnlyEmb
dataset,,,,,
abt-buy_MLP,0.92,0.78,0.41,0.83,0.3
abt-buy_decision tree,0.14,0.78,0.14,0.83,0.45
abt-buy_gaussian naive bayes,0.91,0.93,0.91,0.92,0.6
abt-buy_random forest 100,0.96,0.9,0.53,0.9,0.36
abt-buy_random forest 20,0.92,0.86,0.52,0.85,0.41
abt-buy_random forest 200,0.96,0.9,0.77,0.89,0.34
abt-buy_random forest 50,0.95,0.88,0.28,0.89,0.45
abt-buy_random forest 500,0.96,0.9,0.58,0.9,0.3
abt-buy_svc,0.53,0.76,0.58,0.78,0.4


Mean row wise rank (the lower the better):

In [23]:
rank_order = shallow_expanded.apply(lambda x: x.rank(ascending=False), axis=1).mean().sort_values()
# sort columns by rank to make nemenyi table easier to understand
shallow_expanded = shallow_expanded.reindex(rank_order.index, axis=1)
rank_order

OnlySimNormalized      2.261905
SimAndEmb              2.285714
SimAndEmbNormalized    2.349206
OnlySim                3.388889
OnlyEmb                4.714286
dtype: float64

Interestingly normalization seems to be beneficial here

The Friedman test is designed to check if there is a significant difference in the rank sums.
If this is the case a nemenyi test is applied pairwise to check which differences are significant.

In [24]:
res = statistical_test(shallow_expanded)
res

There is a significant difference in the rank sums:  1.1015122578549892e-28
Significant pairwise differences (p < 0.05) are highlighted in green


,OnlySimNormalized,SimAndEmb,SimAndEmbNormalized,OnlySim,OnlyEmb
OnlySimNormalized,-1,0.9,0.9,0.001,0.001
SimAndEmb,0.9,-1,0.9,0.001,0.001
SimAndEmbNormalized,0.9,0.9,-1,0.00208939,0.001
OnlySim,0.001,0.001,0.00208939,-1,0.001
OnlyEmb,0.001,0.001,0.001,0.001,-1


Normalized Similarity as well as SimAndEmb is significantly better than using only similarities and only embeddings.

## Analysis of Classifier Performance (on SimAndEmb)
Similarly we analyze the different performance of the classifiers. We are interested in the performance using both: similarities and embeddings

In [25]:
whole_df = get_expanded_df(df,"vector_name","SimAndEmb",["embed_model","model_name","dataset"])
stylized = stylize(whole_df)
stylized

,svc,random forest 20,random forest 50,random forest 100,random forest 200,random forest 500,decision tree,gaussian naive bayes,MLP
dataset,,,,,,,,,
D_W_15K_V1_BootEA,0.82,0.81,0.8,0.81,0.82,0.82,0.48,0.83,0.81
D_W_15K_V1_MultiKE,0.87,0.85,0.85,0.86,0.86,0.86,0.79,0.69,0.88
D_W_15K_V1_RDGCN,0.86,0.8,0.81,0.82,0.83,0.83,0.77,0.65,0.82
D_W_15K_V2_BootEA,0.95,0.91,0.92,0.92,0.92,0.92,0.57,0.97,0.95
D_W_15K_V2_MultiKE,0.96,0.93,0.93,0.93,0.93,0.93,0.85,0.67,0.94
D_W_15K_V2_RDGCN,0.9,0.85,0.87,0.87,0.87,0.87,0.84,0.67,0.9
D_Y_15K_V1_BootEA,0.92,0.91,0.93,0.94,0.95,0.96,0.58,0.9,0.94
D_Y_15K_V1_MultiKE,0.99,0.99,0.99,0.99,0.99,0.99,0.97,0.96,0.99
D_Y_15K_V1_RDGCN,0.98,0.99,0.99,0.99,0.99,0.99,0.98,0.95,0.98


In [26]:
rank_order = whole_df.apply(lambda x: x.rank(ascending=False), axis=1).mean().sort_values()
# sort columns by rank to make nemenyi table easier to understand
whole_df = whole_df.reindex(rank_order.index, axis=1)
rank_order

random forest 500       3.119048
random forest 200       3.416667
MLP                     3.630952
random forest 100       3.785714
random forest 50        4.607143
svc                     5.333333
random forest 20        5.916667
gaussian naive bayes    7.071429
decision tree           8.119048
dtype: float64

In [27]:
res = statistical_test(whole_df,features=False)
res

There is a significant difference in the rank sums:  2.130253246450558e-08
Significant pairwise differences (p < 0.05) are highlighted in green


,random forest 500,random forest 200,MLP,random forest 100,random forest 50,svc,random forest 20,gaussian naive bayes,decision tree
random forest 500,-1,0.9,0.9,0.9,0.236666,0.00655798,0.001,0.001,0.001
random forest 200,0.9,-1,0.9,0.9,0.541871,0.0363146,0.001,0.001,0.001
MLP,0.9,0.9,-1,0.9,0.759458,0.101561,0.00415843,0.001,0.001
random forest 100,0.9,0.9,0.9,-1,0.9,0.190561,0.010921,0.001,0.001
random forest 50,0.236666,0.541871,0.759458,0.9,-1,0.9,0.414168,0.00123854,0.001
svc,0.00655798,0.0363146,0.101561,0.190561,0.9,-1,0.9,0.08697,0.001
random forest 20,0.001,0.001,0.00415843,0.010921,0.414168,0.9,-1,0.578132,0.00706359
gaussian naive bayes,0.001,0.001,0.001,0.001,0.00123854,0.08697,0.578132,-1,0.686928
decision tree,0.001,0.001,0.001,0.001,0.001,0.001,0.00706359,0.686928,-1
